In [10]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from xgboost import XGBClassifier
import numpy as np

In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

test_x = test.copy()

In [12]:
train_x = train_x.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
test_x = test_x.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [13]:
for c in ['Sex', 'Embarked']:
    # 学習データにも基づいてどう変換するかを定める 
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    # 学習データ、テストデータを変換 
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

In [14]:
# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(train_x, train_y)

# テストデータの予測値を確率で出力
pred = model.predict_proba(test_x)[:, 1]

# テストデータの予測値を二値に変換
pred_label = np.where(pred>0.5, 1, 0)

# 提出様ファイルの作成
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index=False)

In [16]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

# 各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

# クロスバリデーションを行う
# 学習データを4つに分割しうち１つをバリデーションデータとして使用
"""
n_splits: データをいくつに分けるか指定
shuffle: データセットの中からランダムに値を持ってくる
random_state: 乱数制御パラメータ
"""
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)
    
    # バリデーションの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]
    
    # バリデーションデータでのスコア計算
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    # そのfoldのスコアを保存する
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)
    
# 各foldのスコアの平均を出力する
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')

logloss: 0.4270, accuracy: 0.8148


In [17]:
import itertools

# チューニング候補とするパラメータを準備
param_space = {
    'max_depth': [3,5,7],
    'min_child_weight': [1.0,2.0,4.0]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'],
                                       param_space["min_child_weight"])

# 各パラメータの組み合わせ、それに対するスコア
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーション
for max_depth, min_child_weight in param_combinations:
    
    score_folds = []
    # クロスバリデーションを行う
    # 学習データを４分割してバリデーションデータを変えて繰り返し
    
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth,
                              min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)
        
        # バリデーションの予測値を確率で出力する
        va_pred = model.predict_proba(va_x)[:, 1]
        
        # バリデーションデータでのスコア計算
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)    
        
    # 各foldのスコアを平均
    score_mean = np.mean(score_folds)
    
    # パラメータの組み合わせ、それに対するスコアを保存
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)
    
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

max_depth: 7, min_child_weight: 2.0
